### LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import community
import igraph as ig 
import leidenalg as la 

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.lines import Line2D
from matplotlib.font_manager import FontProperties 
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns

from utils.utils import load_theme, cohesion_metrics
from tqdm import tqdm
import warnings
import ast

warnings.filterwarnings('ignore')

### DATASET

In [2]:
df = pd.read_csv('dataset/transfers_complete.csv')
df = df[df["is_free"] == False]
df = df[df["is_loan"] == False]
df = df[df["is_loan_end"] == False]
df = df[df["is_retired"] == False]
df = df[df["counter_team_name"] != "Without Club"]
transfers = df[['team_name', 'counter_team_name', 'transfer_fee_amnt']]

### GRAPH

In [3]:
G = nx.from_pandas_edgelist(transfers, source='team_name', target='counter_team_name', edge_attr='transfer_fee_amnt', create_using=nx.DiGraph())

degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)

nx.set_node_attributes(G, degree_centrality, 'degree_centrality')
nx.set_node_attributes(G, betweenness_centrality, 'betweenness_centrality')
nx.set_node_attributes(G, eigenvector_centrality, 'eigenvector_centrality')

### METRICS

CLUSTERING

In [4]:
clustering_individual = nx.clustering(G)
sorted_ci = sorted(clustering_individual.items(), key=lambda x:x[1], reverse=True)[:30]
print(sorted_ci)

[('Chelsea FC U18', 1.0), ('Rochdale AFC', 1.0), ('SK Sigma Olomouc', 1.0), ('Chelsea FC Reserves', 1.0), ('Manchester City U18', 1.0), ('Arsenal FC Reserves', 1.0), ('Chunnam Dragons', 1.0), ('Wrexham AFC', 1.0), ('Borussia Dortmund U19', 1.0), ('Crewe Alexandra', 1.0), ('Chesterfield FC', 1.0), ('Aston Villa U21', 1.0), ('Tottenham Hotspur U21', 1.0), ('Falkirk FC', 1.0), ('Rotherham United', 1.0), ('Manchester United U23', 1.0), ('Guangzhou R&F', 1.0), ('IFK Norrköping', 1.0), ('Crystal Palace U21', 1.0), ('Villarreal CF B', 1.0), ('Club Atlético Lanús', 1.0), ('Club Deportivo Godoy Cruz Antonio Tomba', 1.0), ('Real Sociedad B', 1.0), ('União São João Esporte Clube (SP)', 1.0), ('Udinese Primavera', 1.0), ('Real Oviedo', 1.0), ('Wuhan Zall', 1.0), ('Racing Club de Montevideo', 1.0), ('Pomezia Calcio', 1.0), ('AC Isola Liri', 1.0)]


AGGREGAZIONE


In [5]:
transitivity = nx.transitivity(G)
print(transitivity)

0.0943225348273339


DENSITÀ

In [6]:
density = nx.density(G)
print(density)

0.0022821049015041145


ASSORTATIVITÀ

In [7]:
assortativity = nx.degree_assortativity_coefficient(G)
print(assortativity)

0.018044380793770555


CONNESSIONE

! Inutile forse perché il grado è direzionale

In [8]:
connected_components = list(nx.weakly_connected_components(G))
node_connectivity = nx.node_connectivity(G)
edge_connectivity = nx.edge_connectivity(G)

print(connected_components)
print(node_connectivity)
print(edge_connectivity)

[{'Latina Calcio 1932', 'SSD Fidelis Andria', 'Excelsior Rotterdam', 'SC Heerenveen U21', 'Spartak Trnava', 'Renova Dzepciste', 'Estudiantes de Mérida', 'FK Ufa', 'Almere City FC', 'Ital-Lenti Belluno', 'Desportivo Aves', 'AS Monaco U19', 'AFC Bournemouth U18', 'FC Ingolstadt 04', 'Genoa CFC', 'Benfica Luanda', 'UD Tocha', 'New York City FC', 'RC Strasbourg Alsace', 'SK Austria Klagenfurt', 'FC Girondins Bordeaux', 'Club Guaraní', 'SO Cholet', 'FC Stade Rennes B', 'Chamois Niortais FC', 'Rio Claro Futebol Clube (SP)', 'CD Aves B', 'Podbeskidzie Bielsko-Biala', 'Città di Gela', 'Berço SC', 'FC Slovan Liberec', 'Atlético de Rafaela', '1.FC Magdeburg', 'KF Vushtrria', 'AZ Alkmaar U19', 'Clube Atlético Bragantino (SP)', 'Portsmouth FC U18', 'Lommel SK', 'FC Nantes B', 'FK Proleter Novi Sad', 'Vitória Guimarães SC B', 'Cercle Brugge', 'FC Grosseto', 'JEF United Chiba', 'Grasshopper Club Zürich U18', 'PSV Eindhoven U19', 'FC Timisoara', 'Makedonija Gjorce Petrov', 'Toulouse FC B', 'Sunderlan

MODULARITÀ (Comunità)

! Errore: grado diretto non idoneo

In [9]:
central_nodes = sorted(degree_centrality, key=degree_centrality.get, reverse=True)[:100]
print(central_nodes)

subgraph = G.subgraph(central_nodes)
sub_und = subgraph.to_undirected()

partition = community.best_partition(sub_und)

communities = {}
for node, community_id in partition.items():
    if community_id not in communities:
        communities[community_id] = []
    communities[community_id].append(node)

print(communities)

['Udinese Calcio', 'SL Benfica', 'Genoa CFC', 'AS Roma', 'ACF Fiorentina', 'Sporting CP', 'Sevilla FC', 'SSC Napoli', 'Atalanta BC', 'VfL Wolfsburg', 'FC Porto', 'AS Monaco', 'Juventus FC', 'Stade Rennais FC', 'Ajax Amsterdam', 'Manchester City', 'Chelsea FC', 'SC Braga', 'LOSC Lille', 'Bologna FC 1909', 'Torino FC', 'Parma FC', 'Vitória Guimarães SC', 'PSV Eindhoven', 'OGC Nice', 'Olympique Marseille', 'AZ Alkmaar', 'UC Sampdoria', 'TSG 1899 Hoffenheim', 'Olympique Lyon', 'SS Lazio', 'VfB Stuttgart', 'Inter Milan', 'AC Milan', 'Chievo Verona', 'Atlético de Madrid', 'Tottenham Hotspur', 'Eintracht Frankfurt', 'West Ham United', 'Arsenal FC', 'Liverpool FC', 'Bayer 04 Leverkusen', 'Hellas Verona', 'Paris Saint-Germain', 'US Palermo', 'FC Toulouse', 'Valencia CF', 'Aston Villa', 'Cagliari Calcio', 'Villarreal CF', '1.FSV Mainz 05', 'SV Werder Bremen', 'US Sassuolo', 'AS Saint-Étienne', 'Newcastle United', 'FC Groningen', 'CS Marítimo', 'Borussia Dortmund', 'FC Schalke 04', 'Wolverhampton

In [10]:
g_undirected = G.to_undirected()
partition_g = community.best_partition(g_undirected)
modularity = community.modularity(partition_g, g_undirected)
print(modularity)


0.4441735067574498


In [11]:
modularity = community.modularity(partition, sub_und)
print(modularity)

0.21011309939086234


### FUNCTION

In [12]:
print(cohesion_metrics(G))

{'transitivity': 0.0943225348273339, 'density': 0.0022821049015041145, 'assortativity': 0.018044380793770555, 'modularity': 0.4259074571664968, 'clustering': [('Chelsea FC U18', 1.0), ('Rochdale AFC', 1.0), ('SK Sigma Olomouc', 1.0), ('Chelsea FC Reserves', 1.0), ('Manchester City U18', 1.0), ('Arsenal FC Reserves', 1.0), ('Chunnam Dragons', 1.0), ('Wrexham AFC', 1.0), ('Borussia Dortmund U19', 1.0), ('Crewe Alexandra', 1.0)]}
